# Quantitative Analysis for Complex QA

## Negation terms

Reference: https://github.com/VandanaYadav24/NLP_NegationDetection_SentimentAnalysis/tree/main

In [1]:
!pip install thefuzz

In [2]:
!pip install textblob

In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
from nltk import pos_tag, word_tokenize, RegexpParser
from nltk.chunk import ne_chunk

from thefuzz import fuzz

import string
import re

from textblob import TextBlob

In [2]:
# Fuzzywuzzy string matching algorithm for identifying operating negations in the given text
def identify_negative_words(input_text_tokens):
    negations = ['no', 'none', 'not', 'neither', 'nor', 'never', "nothing", "nobody", "nowhere"]
    a = []
    idx_text = []
    negation_words = []
    for i in range(len(negations)):
        a.append([])
        idx_text.append([])
        for j in range(len(input_text_tokens)):
            a[i].append(fuzz.ratio(negations[i], input_text_tokens[j]))
        idx_text[i] = [idx for idx, x in enumerate(a[i]) if x==100]
    for m in range(len(idx_text)):
        if idx_text[m] != []:
            [negation_words.append(input_text_tokens[t]) for t in idx_text[m]]
    return negation_words

# Regular expressions to identify prefix, suffix type negations
def identify_prefix_neg_words(input_text_tokens):
    prefix_negs = ['un', 'im', 'in', 'il', 'ir', 'dis', 'non', 'anti', 'de', 'counter', 'mal']
    prefix_neg_words = []
    idx_text = []
    for i in range(len(prefix_negs)):
        idx_text.append([])
        for j in range(len(input_text_tokens)):
            pattern = "^%s[a-z]+$" %prefix_negs[i]
            blob=TextBlob(input_text_tokens[j])
            if (re.search(pattern, input_text_tokens[j]) and blob.sentiment.polarity<0):
                idx_text[i].append(j)
    for m in range(len(idx_text)):
        if idx_text[m] != []:
            [prefix_neg_words.append(input_text_tokens[t]) for t in idx_text[m]]
    return prefix_neg_words

#identify suffix_negations in the text
def identify_suffix_neg_words(input_text_tokens):
    suffix_negs = ['less']
    suffix_neg_words = []
    idx_text = []
    for i in range(len(suffix_negs)):
        idx_text.append([])
        pattern = "^[a-z]+%s$" %suffix_negs[i]
        for j in range(len(input_text_tokens)):
            blob=TextBlob(input_text_tokens[j])
            if (re.search(pattern, input_text_tokens[j]) and blob.sentiment.polarity<=0):
                idx_text[i].append(j)
    for m in range(len(idx_text)):
        if idx_text[m] != []:
            [suffix_neg_words.append(input_text_tokens[t]) for t in idx_text[m]]
    return suffix_neg_words

# Convert "n't" to "not"
def decontract(phrase):
    phrase = re.sub(r"[c|C]an\'t", "can not", phrase)
    phrase = re.sub(r"[w|W]on\'t", "will not", phrase)
    
    phrase = re.sub(r"n\'t", " not", phrase)
    return phrase

# Consolidating 3 methods to identify operating, preffix, suffix type negations in one method 
def identify_all_negs(text):
    tokens_text = word_tokenize(text)
    lowercase_tokens_text = [t.lower() for t in tokens_text]
    negation_words = identify_negative_words(lowercase_tokens_text)
    prefix_negation_words = identify_prefix_neg_words(lowercase_tokens_text)
    suffix_negation_words = identify_suffix_neg_words(lowercase_tokens_text)
    all_negs = negation_words+ prefix_negation_words+ suffix_negation_words
    #return negation_words, prefix_negation_words, suffix_negation_words, all_negs
    return all_negs

## Hedge terms

In [3]:
# Fuzzywuzzy string matching algorithm for identifying hedge terms in the given text
def identify_hedge_words(input_text_tokens):
    hedges = ["believe", "maybe", "likely", "probably", "possibly", "putative", 
              "seem to be", "appear to be", "may be", "could be", "might be", "tend to be",
              "suggest", "imply", "indicate", "conceivably", "doubt", "presumably",
              "often", "sometimes", "usually", "generally"]
    a = []
    idx_text = []
    hedge_words = []
    for i in range(len(hedges)):
        a.append([])
        idx_text.append([])
        for j in range(len(input_text_tokens)):
            a[i].append(fuzz.ratio(hedges[i], input_text_tokens[j]))
        idx_text[i] = [idx for idx, x in enumerate(a[i]) if x==100]
    for m in range(len(idx_text)):
        if idx_text[m] != []:
            [hedge_words.append(input_text_tokens[t]) for t in idx_text[m]]
    return hedge_words

def identify_all_hedges(text):
    tokens_text = word_tokenize(text)
    lowercase_tokens_text = [t.lower() for t in tokens_text]
    hedge_words = identify_hedge_words(lowercase_tokens_text)
    return hedge_words

## Prepositional phrases extraction

Reference: https://github.com/merionum/pphrase

In [6]:
!pip install ufal.udpipe


In [4]:
from ufal.udpipe import Model, Pipeline
import networkx as nx
from argparse import ArgumentParser
import json
from tqdm import tqdm
import pandas as pd

In [5]:
class Token():

    def __init__(self, token: list):
        self.id = int(token[0])
        self.form = token[1]
        self.lemma = token[2]
        self.upos = token[3]
        self.xpos = token[4]
        self.feats = token[5]
        self.head = int(token[6])
        self.dep = token[7]
        self.misc = token[8]

def get_preps(sent):
    preps = dict()
    fixed = set()
    for tok in sent:
        if not tok.dep =='fixed':
            continue
        if tok.head not in preps:
            preps[tok.head] = [[tok]]
        else:
            preps[tok.head][0].append(tok)
        fixed.add(tok)

    for tok in sent:
        if tok.id in preps:
            preps[tok.id][0].append(tok)
            preps[tok.id].append(tok.head)
        elif tok.upos == 'ADP' and tok not in fixed:
            preps[tok.id] = [[tok], tok.head]

    preps = {k: v for k, v in preps.items()
             if any([tok.upos == 'ADP' for tok in v[0]])}
    return preps

def get_succesors_by_id(sent, id):
    G = nx.DiGraph()
    G.add_edges_from([(tok.head, tok.id) for tok in sent])
    return nx.dfs_successors(G, source=id)

def get_sorted_sent(tokens):
    return ' '.join([tok.form for tok in sorted(tokens,
                     key=lambda x: x.id, reverse=False)])

def get_tok_tags(token):
    return f'PartOfSpeech={token.upos}|{token.feats}'

def get_phrase(prep, dep_id, sent):
    dep = sent[dep_id-1]
    host = sent[dep.head-1]
    bad_host = False
    members = [dep, host]
    if host == dep or host.upos == 'PUNCT':
        bad_host = True
        members.pop()

    prep_ids = {p.id for p in prep}
    members_ids = {t.id for t in members}

    dep_succesors = get_succesors_by_id(sent, dep.id)

    dep_subtree = [sent[i-1] for i in set().union(*dep_succesors.values())
                                   if i not in prep_ids|members_ids]

    phrase = {'phrase': get_sorted_sent(prep + dep_subtree + members),
              'host': host.form if not bad_host else None,
              'preposition': get_sorted_sent(prep),
              'dependant': dep.form,
              'full_dependant': get_sorted_sent([dep] + dep_subtree),
              'host_tags': get_tok_tags(host) if not bad_host else None,
              'dependant_tags': get_tok_tags(dep),
              'host_lemma': host.lemma if not bad_host else None,
              'dependant_lemma': dep.lemma}

    return phrase

def run(model_file, text_file):
    print('Loading model...')
    model = Model.load(model_file)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT,
                                Pipeline.DEFAULT, 'conllu')
    #print('Reading corpus...') #!
    #with open(text_file) as f: #!
    #    text = f.read() #!

    text = text_file #!
    
    print('Analyzing text...') #!
    processed = pipeline.process(text)

    print('Extracting phrases...') #!
    phrases = []
    sent = []
    for line in tqdm((processed+'#').splitlines()):
        if line.startswith('#') and len(sent):
            preps = get_preps(sent)
            for prep, dep_id in preps.values():
                pphrase = get_phrase(prep, dep_id, sent)
                phrases.append(pphrase)
            sent.clear()
        elif len(line) > 1:
            try:
                sent.append(
                        Token(
                            line.split('\t')
                        )
                )
            except ValueError:
                continue
    print('Done!') #!
    return phrases

## Analysis

In [6]:
# Import libraries
import json
import pandas as pd
import numpy as np
import math

In [8]:
# Read the pmpqa dataset statements (jsonl)

filename = 'merged'
data = []
with open(filename + '.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

print('samples',len(data))
print('properties',len(data[0]))
print('no. statements',len(data[0]['statements']))
print('total no. statements',len(data)*len(data[0]['statements']))

samples 2757
properties 8
no. statements 4
total no. statements 11028


In [9]:
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Negation Terms detection
%time
neg_terms = []
for idx in range(len(data)):
    negs = []
    for statements in data[idx]['statements']:
        input_text = statements['statement']
        negs.extend(identify_all_negs(input_text))
    if negs:
        neg_terms.append(1)
    if not negs:
        neg_terms.append(0)    

CPU times: total: 0 ns
Wall time: 0 ns


In [11]:
# Hedge Terms detection
%time
hedge_terms = []
for idx in range(len(data)):
    hedges = []
    for statements in data[idx]['statements']:
        input_text = statements['statement']
        hedges.extend(identify_all_hedges(input_text))
    #print(hedges)
    if hedges:
        hedge_terms.append(1)
    if not hedges:
        hedge_terms.append(0)

CPU times: total: 0 ns
Wall time: 0 ns


In [12]:
# Propositional Phrases detection
%timeit
model_file = "english-ewt-ud-2.5-191206.udpipe"
p_phrases = []
for idx in range(len(data)):
    input_text = data[idx]['question']['stem']
    phrases = run(model_file, input_text)
    count = len(phrases)
    p_phrases.append(count)

Loading model...
Analyzing text...
Extracting phrases...


100%|██████████| 25/25 [00:00<00:00, 24960.15it/s]

Done!
Loading model...
Analyzing text...
Extracting phrases...



100%|██████████| 17/17 [00:00<?, ?it/s]


Done!
Loading model...
Analyzing text...
Extracting phrases...


...

100%|██████████| 30/30 [00:00<00:00, 30037.99it/s]

Done!
Loading model...


Analyzing text...
Extracting phrases...


100%|██████████| 30/30 [00:00<00:00, 29838.54it/s]

Done!


In [13]:
# Quantitative Analysis dataframe

qa_df = pd.DataFrame(data={
                           "negation": neg_terms,
                           "hedge": hedge_terms,
                           "p_phrases": p_phrases,
                          })

#qa_df = pd.concat([qa_df,preds], axis=1)
print(qa_df.dtypes)
qa_df

negation     int64
hedge        int64
p_phrases    int64
dtype: object


,negation,hedge,p_phrases
0,0,0,2
1,1,0,2
2,1,0,2
3,1,0,2
4,0,0,3
...,...,...,...
2752,1,0,2
2753,1,0,2
2754,0,0,2
2755,0,0,4


In [14]:
# save the table with analysis

qa_df.to_csv(filename + '.csv')